In [1]:
import numpy as np 
import functools
from scipy import sparse
from scipy.sparse import linalg as slinalg
import pickle
from time import time
import numpy.linalg as npl
import cProfile
import matplotlib.pyplot as plt
import random as ran

sites = 12


def SaveData(data, saveas): 
    pickle.dump(data, open(saveas, 'wb'), protocol = pickle.HIGHEST_PROTOCOL)

def LoadData(filename):
    return pickle.load(open(filename, 'rb'))

def tensorproduct(*M, use_sparse = False):
	if not use_sparse:
		return functools.reduce(lambda x, y: np.kron(x, y), M)
	else:
		return functools.reduce(lambda x, y: sparse.kron(x, y), M)


def sparse_multi_dot(*Ms):
	return functools.reduce(lambda x, y: sparse.csr_matrix.dot(x, y), Ms)

#Here I want to generate entanglement spectra of arbitrary systems

"""
our total system will be a quantum spin chain of 12 spins

The whole construction, I work with sparse matrices

we will separate it into 2 subsystems, A and B

Subsystem A will be the middle quarter of each eigenvector

Subsystem B will be the rest of each eigenvector

"""

make_sigmas = True

if make_sigmas:

	def make_sigma(i, pauli_mat):
		#i ranges from 1 to 12 (for 12 total sites)
		#pauli is 1,2,or 3
		if i>12:
			return np.zeros([2**12, 2**12])

		s1 = [[0., 1.], [1., 0.]]
		s2 = [[0., -1j], [1j, 0.]]
		s3 = [[1., 0], [0., -1.]]

		if pauli_mat == 1:
			s = s1
		elif pauli_mat ==2:
			s = s2
		else:
			s = s3

		ident = [[1., 0.], [0., 1.]]

		tensorthis = []

		for j in range(sites):
			if j+1 == i:
				tensorthis.append(s)
			else:
				tensorthis.append(ident)

		out = tensorproduct(*tensorthis)

		return out

	sigmas_x = [sparse.csr_matrix(make_sigma(i, 1)) for i in range(1, 13)]
	
	sigmas_y = [sparse.csr_matrix(make_sigma(i, 2)) for i in range(1, 13)]
	
	sigmas_z = [sparse.csr_matrix(make_sigma(i, 3)) for i in range(1, 13)]

	tosave = (sigmas_x, sigmas_y, sigmas_z)

	
	print(sigmas_y[0].shape)
	

	SaveData(tosave, "sparse_sigmas.pkl")

(4096, 4096)


In [2]:
#Mersenne Twister random number generator couplings (from 0 to 1) is what appears below
J_list = []
h_list = []
L_list = []
for i in range(0,13):
    J_list.append(ran.random())
    h_list.append(ran.random())
    L_list.append(ran.random())
J_list[1]

0.3329506875938647

In [5]:
t = time()
sigmas = LoadData("sparse_sigmas.pkl")
print("loading sigmas: ", time()-t)

def sigma(i, pauli_mat):
	if i>12:
			return sparse.csr_matrix(np.zeros([2**sites, 2**sites]))

	return sigmas[pauli_mat-1][i-1]


def h_term(i, J_list, h_list, L_list):
    if i< sites:
        term = J_list[i]*sparse.csr_matrix.dot(sigma(i, 1), sigma(i+1, 1))
        term += h_list[i]*sigma(i, 3)
        term += L_list[i]*(sparse.csr_matrix.dot(sigma(i, 2), sigma(i+1, 2))+sparse.csr_matrix.dot(sigma(i, 3), sigma(i+1, 3)))
    elif i == sites:
        term = J_list[i]*sparse.csr_matrix.dot(sigma(i, 1), sigma(0, 1))
        term += h_list[i]*sigma(i, 3)
        term += L_list[i]*(sparse.csr_matrix.dot(sigma(i, 2), sigma(0, 2))+sparse.csr_matrix.dot(sigma(i, 3), sigma(0, 3)))
    else:
        None

    return -term


"""
# test to show that the sparse matrix math produces identical output

m1 = np.random.rand(100, 100)
m2 = np.random.rand(100, 100)

sm1 = sparse.csr_matrix(m1)
sm2 = sparse.csr_matrix(m2)

out1 = np.matmul(m1, m2)
out2 = sparse.csr_matrix.dot(sm1, sm2)
out2 = out2.todense()
#we're good
"""


def Hamiltonian(J_list, h_list, L_list, showtime = False):
	t=time()
	H = sparse.csr_matrix(np.zeros([2**sites, 2**sites]))
	for i in range(1, sites+1):

		H += h_term(i, J_list, h_list, L_list)

	if showtime:
		print("Hamiltonian build time: ", -t+time())

	return H

def Ham_Spec_Sim(iterr, k):
    #This function gives us the eigenvalues for a specific list of couplings
    #k is number of eigenvalues you want per coupling, iterr is the number of times a random J_list is created
    ee = []
    ex = []
    h_list = []
    L_list = []
    for j in range(0,sites+1):
        #create lists of random couplings--change later to input whatever is needed
        h_list.append(ran.random()/5)
        L_list.append(ran.random()/10)
    for i in range(iterr):
        #initialize
        J_list = []
        for j in range(0,sites+1):
            #create lists of random couplings
            J_list.append(ran.normalvariate(0,.8))
        mean_J = np.mean(J_list)
        b = slinalg.eigs(Hamiltonian(J_list,h_list,L_list), k)[0]
        ee.append(b)
        ex.append([mean_J]*len(b))
    return ex, ee
                  
Ham_Spec_Sim(1,2)[0]

loading sigmas:  0.0039899349212646484


[[-0.35874873286253206, -0.35874873286253206]]

In [ ]:
a = Ham_Spec_Sim(200,200)[0]
b = Ham_Spec_Sim(200,200)[1]

In [ ]:
plt.scatter(a,b,s=5,)
plt.xlabel('J avg')
plt.ylabel('E')
plt.show()

In [ ]:
#Marginalize over a single spinor on the left or the right of the system

def marginalize_single_spin(density_matrix, first_spin = True):
	#density matrix is a sparse matrix

	nextsize = int(density_matrix.shape[0]/2)

	ident = sparse.identity(nextsize, format = "csr")

	b1 = sparse.csr_matrix([[1.],[0.]])
	b2 = sparse.csr_matrix([[0.],[1.]])

	basis = [b1, b2]
	out = 0*ident 
	
	for b in basis:
		temp = tensorproduct(b, ident, use_sparse = True) if first_spin else tensorproduct(ident, b, use_sparse=True)
		
		out+=sparse_multi_dot(temp.T, density_matrix, temp)
		
	return out

#Use the previous function n_first times on the left and n_last times on the right

def marginalize(density_matrix, first = 4, last = 4):
	#return partial trace density matrix

	for i in range(first):
		density_matrix = marginalize_single_spin(density_matrix, first_spin=True)

	for i in range(last):
		density_matrix = marginalize_single_spin(density_matrix, first_spin=False)

	return density_matrix

#Do this in jupyter

#given a state phi, lets try to use svd
#suppose i split it into na, nb. 
#then i'll have 
#Dim_a = 2^na, Dim_b =2^nb
#both are pretty sparse i guess
#i could build them once, or create a fast function to make them, 
#since i wouild be dealing with just 1's and 0's

#then ill have to go through all 2^(na+nb) combinations
#this is the length of the vector

#then I would have to do my svd
#it turns out that svd's are really fast (and in c)

#if the first part doesnt take too mmuch time, i think i can get this much, much faster

def give_basis(index_a, index_b, left = 4, right= 4):
    #index_a, index_b gives us the index for the subsystem we will be 
    #marginalizing over/keeping 
    
    va = "{0:b}".format(index_a)[::-1]
    vb = "{0:b}".format(index_b)[::-1]
    
    va+='0'*((left+right)-len(va))
    vb+='0'*(12-(left+right)-len(vb))
    
    #now insert vb into the appropriate location in va
    binary_vector = va[:left]+vb+va[left:]
    
    index = int(binary_vector, 2)
    
    return (2**12 -1) - index
        
 
def give_spectrum_1(eigenvector, left = 4, right = 4):
    #eigenvector is an array of shape (2**12,)
    #marginalize over A to obtain the spectrum for B
    n_a = left+right #number of spinors in A
    n_b = 12 - left-right #number of spinors in B
    
    #size of the two bases
    basis_a_size = 2**n_a
    basis_b_size = 2**n_b
    
    out_matrix = np.zeros((basis_a_size, basis_b_size))
    
    #now we iterate through the two bases
    
    for index_a in range(basis_a_size):
        for index_b in range(basis_b_size):
            out_matrix[index_a, index_b] = eigenvector[give_basis(index_a, index_b)]
    
    s = npl.svd(out_matrix, compute_uv=False)
    
    return np.sort(-np.log(s**2))







def give_spectrum(eigenvector):
	density_matrix = sparse.csr_matrix(np.outer(eigenvector, np.conj(eigenvector)))

	reduced_density_matrix = marginalize(density_matrix)

	#now i want all eigenvalues of this matrix

	vals, vecs = npl.eigh(reduced_density_matrix.todense())

	return np.sort(-np.log(vals))


"""v = np.random.rand(2**12)
v = v/np.sqrt(np.dot(v,v))

x = np.sort(give_spectrum_1(v))

y = np.sort(give_spectrum(v))


#make these equal

print(x-y)
print(" ")
print(x)
print(" ")
print(y)
"""











def make_datum(mean_log_J, mean_log_h, L, take_middle = .8, save = True, \
	nametag = '',lowerhalf = False, vocal = True):
	#here we take the middle 80% of eigenvectors for a single disorder config, and we marginalize over the outer sections of the system
	print("mean_log_J-mean_log_h: ", mean_log_J-mean_log_h)
	print("lambda: ", L)

	spectra = []

	t = time()#time the entire process

	h = Hamiltonian(mean_log_J, mean_log_h, L)
	n = h.shape[0]

	#how long does this take
	t1 = time()
	vals, vecs = slinalg.eigsh(h, k = round(n*(.5+take_middle/2)))
	if vocal:
		print("diagonalization time: ", time()-t1)
		print("beginning spectra calculation")
	t2 = time()
	for i in range(vecs.shape[1]):
		if round(n*(.5-take_middle/2)) <= i:
			evec = vecs[:, i]
			#now we run the procedure to obtain the entangelement spectrum

			s = give_spectrum_1(evec)
			
			if lowerhalf:
				half = len(s)//2
				s = s[:half]

			spectra.append(s)

	if vocal:
		print("spectra calc time: ",time()-t2)
		print("Total time: ", time()-t)

	if save:
		np.save("spectra_mlj"+str(mean_log_J)+"_mlh_"+str(mean_log_h)+"_l_"+str(L)+'tag'+str(nametag), spectra) 

	return spectra


def make_data(num_samples, mean_log_J, mean_log_h, L, take_middle = .8, timeit = True,\
	 lowerhalf=False, nametag = ''):
	#we now run make_datum 'num_samples' times
	t1 = time()
	spectra_data = []
	

	
	for _ in range(num_samples):
		spectrum = make_datum(mean_log_J, mean_log_h, L, take_middle = take_middle, lowerhalf=lowerhalf, save = False, vocal=False)
		spectra_data.append(spectrum)
	
	

	#SaveData(spectra_data, "Entanglement_Spectra_"+"mlj_"+str(mean_log_J)+"_mlh_"+str(mean_log_J)+"_l_"+str(mean_log_J))
	#for now i'll use numpy's save
	
	
	np.save("/u/flashscratch/s/stevendu/Rigorous_Entanglement_Spectra_"+"mlj_"+str(mean_log_J)+"_mlh_"+str(mean_log_h)+"_l_"+str(L)+"_num_"+str(num_samples)+'_tag_'+str(nametag), spectra_data)
	#np.save("u/flashscratch/s/stevendu/Rigorour_Entanglement_Spectra_"+"mlj_"+str(mean_log_J)+"_mlh_"+str(mean_log_h)+"_l_"+str(L)+"_num_"+str(num_samples)+'_tag_'+str(nametag), spectra_data)

	if timeit:
		print("Total Time: ",time()-t1)

In [4]:
#Mersenne Twister random number generator couplings (from 0 to 1)
J_list = []
h_list = []
L_list = []
for i in range(0,13):
    J_list.append(ran.random())
    h_list.append(ran.random())
    L_list.append(ran.random())
J_list[1]

0.02001328579303996